# IBM Data Science Capstone Project by Coursera

<h3>Week 5 Final report</h3>

<h4>Opening a new shopping mall in Amsterdam

<h3>1. Importing libraries required

import pandas as pd

In [14]:
!pip install geocoder
!pip install folium
body = client_fb6819e4948347ffbc27a58a46403600.get_object(Bucket='clusteringareasbasedonhouseprices-donotdelete-pr-uux78lztes1x6h',Key='Amsterdam_Neighbourhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:01
     |████████████████████████████████| 92kB 2.9MB/s eta 0:00:01


NameError: name 'client_fb6819e4948347ffbc27a58a46403600' is not defined

In [15]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates
import folium # map rendering library
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

<h3>2. Import csv file of list of neighborhoods from Assets folder

In [16]:
# The code was removed by Watson Studio for sharing.

,Neighbourhoods,Latitude,Longitude
0,Admiralenbuurt,52.372734,4.856363
1,Amsteldorp,52.360540,4.905160
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030


In [17]:
df_data_2.shape

(105, 3)

In [18]:
df_data_2 = df_data_2.dropna()

In [19]:
df_data_2.shape

(105, 3)

<h3>3. Get latitude and longitude of Amsterdam city from geocode package

In [20]:
address = "Amsterdam, Netherlands"
geolocator = Nominatim(user_agent="amsterdam_data")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude and longitude of Amsterdam are {},{}".format(latitude,longitude))

Latitude and longitude of Amsterdam are 52.3727598,4.8936041


<h3>4. Create a Map of Amsterdam city with neighborhoods superimposed on top

In [21]:
am_map = folium.Map(location=[latitude,longitude],zoom_start=10)
for lat,lon,label in zip(df_data_2.Latitude,df_data_2.Longitude,df_data_2.Neighbourhoods):
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(am_map)
am_map

<h3>5. Use Foursquare API to get venues in each neighborhood

In [24]:
CLIENT_ID = '5L5BY3RY1JEIXF2BT5BVI3SD334AD1YORIGWC1G1K53WZ1QT' # your Foursquare ID
CLIENT_SECRET = 'Z4C0MYGGAUTVLWOCE1RQRW3WT1YVA0UCLXKGF5UDZQZS52QB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5L5BY3RY1JEIXF2BT5BVI3SD334AD1YORIGWC1G1K53WZ1QT
CLIENT_SECRET:Z4C0MYGGAUTVLWOCE1RQRW3WT1YVA0UCLXKGF5UDZQZS52QB


<h4>Now lets get top 100 venues in each neighborhood within radius of 500 meters.

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
amsterdam_venues = getNearbyVenues(df_data_2['Neighbourhoods'],df_data_2['Latitude'],df_data_2['Longitude'])

Admiralenbuurt
Amsteldorp
Amsterdam Oud-West
Amsterdam Oud-Zuid
Amsterdam Science Park
Apollobuurt
Betondorp
Bijlmermeer
Binnenstad
Bos en Lommer
Buiksloot
Buikslotermeer
Buitenveldert
Bullewijk
Burgwallen Nieuwe Zijde
Burgwallen Oude Zijde
Chassébuurt
Cruquiuseiland
Czaar Peterbuurt
Dapperbuurt
De Aker
De Pijp
De Wallen
Diamantbuurt
Duivelseiland
Eastern Docklands
Eendracht
Floradorp
Frederik Hendrikbuurt
Gaasperdam
Geuzenveld
Gouden Reael
Grachtengordel
Haarlemmerbuurt
Hoofddorppleinbuurt
Houthaven
IJburg
Indische Buurt
Java-eiland
Jodenbuurt
Jordaan
Kadijken
Kadoelen
Kinkerbuurt
KNSM Island
Kolenkit District
Landelijk Noord
Landlust
Lastage
Middelveldsche Akerpolder
Molenwijk
Museumkwartier
NDSM
Negen Straatjes
Nieuw Sloten
Nieuwe Pijp
Nieuwendam
Nieuwendammerdijk en Buiksloterdijk
Olympisch Kwartier
Omval
Oostelijke Eilanden
Oosterdokseiland
Oosterparkbuurt
Oostoever
Oostpoort
Oostzanerwerf
Osdorp
Oud Osdorp
Oud-Oost
Oude Pijp
Overhoeks
Overtoombuurt
Overtoomse Veld
Plantage
Prinse

In [13]:
amsterdam_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Admiralenbuurt,52.372734,4.856363,Radijs,52.371049,4.856756,Bistro
1,Admiralenbuurt,52.372734,4.856363,Sapporo Ramen Sora,52.371294,4.855144,Ramen Restaurant
2,Admiralenbuurt,52.372734,4.856363,Deli-caat,52.371221,4.856056,Deli / Bodega
3,Admiralenbuurt,52.372734,4.856363,Café Cook,52.371208,4.852792,Pub
4,Admiralenbuurt,52.372734,4.856363,Maz Mez,52.371231,4.857968,Lebanese Restaurant


In [14]:
amsterdam_venues.shape

(4346, 7)

<h4>Let's check how many venues were returned for each neighborhood.

In [15]:
amsterdam_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Admiralenbuurt,44,44,44,44,44,44
Amsteldorp,66,66,66,66,66,66
Amsterdam Oud-West,100,100,100,100,100,100
Amsterdam Oud-Zuid,56,56,56,56,56,56
Amsterdam Science Park,4,4,4,4,4,4
Apollobuurt,33,33,33,33,33,33
Betondorp,9,9,9,9,9,9
Bijlmermeer,4,4,4,4,4,4
Binnenstad,95,95,95,95,95,95


In [16]:
print('There are {} unique categories'.format(len(amsterdam_venues['Venue Category'].unique())))

There are 293 unique categories


In [17]:
amsterdam_venues['Venue Category'].unique()

array(['Bistro', 'Ramen Restaurant', 'Deli / Bodega', 'Pub',
       'Lebanese Restaurant', 'Spa', 'Pet Café', 'Tea Room',
       'Grocery Store', 'Caribbean Restaurant',
       'Middle Eastern Restaurant', 'Gym / Fitness Center', 'Café',
       'Park', 'Coffee Shop', 'Restaurant', 'Bar', 'Falafel Restaurant',
       'Dance Studio', 'Scandinavian Restaurant', 'Beer Garden',
       'Gift Shop', 'Clothing Store', 'Toy / Game Store',
       'Italian Restaurant', 'Fish Market', 'Food & Drink Shop',
       'Supermarket', 'Greek Restaurant', 'Snack Place', 'Massage Studio',
       'Bagel Shop', 'Fast Food Restaurant', 'Plaza', 'Ice Cream Shop',
       'Pizza Place', 'Chinese Restaurant', 'Wine Shop', 'Bakery',
       'Hotel', 'Theater', 'Yoga Studio', 'French Restaurant',
       'Dessert Shop', 'Hotel Bar', 'Steakhouse', 'Pool Hall',
       'Indie Movie Theater', 'College Arts Building',
       'Marijuana Dispensary', 'Spanish Restaurant', 'Hostel',
       'Cheese Shop', 'Japanese Restaurant'

In [18]:
"Shopping Mall" in amsterdam_venues['Venue Category'].unique()

True

<h3>6. Analyze each neighborhood

In [29]:
amd_onehot = pd.get_dummies(amsterdam_venues['Venue Category'])
amd_onehot['Neighborhoods'] = amsterdam_venues['Neighborhood']
finalcolumns = [amd_onehot.columns[-1]] + list(amd_onehot.columns[:-1])
amd_onehot = amd_onehot[finalcolumns]
print(amd_onehot.shape)
amd_onehot.head()

(4402, 295)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Admiralenbuurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Admiralenbuurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Admiralenbuurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Admiralenbuurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Admiralenbuurt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Now lets group rows by neighborhood and by taking the mean of the frequency of occurence of each category.

In [31]:
amd_grouped = amd_onehot.groupby('Neighborhoods').mean().reset_index()
print(amd_grouped.shape)
amd_grouped.head()

(105, 295)


,Neighborhoods,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Admiralenbuurt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.00,0.022222,0.0,0.000000,0.0,0.0
1,Amsteldorp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.014706,0.0,0.0
2,Amsterdam Oud-West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.01,0.0,0.0,0.01,0.000000,0.0,0.020000,0.0,0.0
3,Amsterdam Oud-Zuid,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035088,...,0.0,0.00,0.0,0.0,0.00,0.017544,0.0,0.000000,0.0,0.0
4,Amsterdam Science Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0


In [32]:
amd_shoppingmall = amd_grouped[['Neighborhoods','Shopping Mall']]
amd_shoppingmall.head()

,Neighborhoods,Shopping Mall
0,Admiralenbuurt,0.0
1,Amsteldorp,0.0
2,Amsterdam Oud-West,0.0
3,Amsterdam Oud-Zuid,0.0
4,Amsterdam Science Park,0.0


In [33]:
len(amd_shoppingmall[amd_shoppingmall['Shopping Mall']>0])

11

<h3>Cluster Neighbourhoods</h3>

<h5>Run kmeans and cluster neighborhoods in amsterdam city into 3 clusters</h5>

In [34]:
from sklearn.cluster import KMeans

In [35]:
#k-means
amd_custering = amd_shoppingmall.drop('Neighborhoods',axis=1)
k_clusters = 3
#initialize kmeans
kmeans = KMeans(n_clusters=k_clusters,random_state=0)
kmeans.fit(amd_custering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [36]:
amd_merged = amd_shoppingmall.copy()
amd_merged['Cluster labels'] = kmeans.labels_
amd_merged.head()

,Neighborhoods,Shopping Mall,Cluster labels
0,Admiralenbuurt,0.0,0
1,Amsteldorp,0.0,0
2,Amsterdam Oud-West,0.0,0
3,Amsterdam Oud-Zuid,0.0,0
4,Amsterdam Science Park,0.0,0


In [37]:
amd_merged.rename(columns={'Neighborhoods':'Neighbourhoods'},inplace=True)
amd_merged.head()

,Neighbourhoods,Shopping Mall,Cluster labels
0,Admiralenbuurt,0.0,0
1,Amsteldorp,0.0,0
2,Amsterdam Oud-West,0.0,0
3,Amsterdam Oud-Zuid,0.0,0
4,Amsterdam Science Park,0.0,0


In [38]:
amd_merged = amd_merged.join(df_data_2.set_index('Neighbourhoods'),on='Neighbourhoods')
print(amd_merged.shape)
amd_merged.head()

(105, 5)


,Neighbourhoods,Shopping Mall,Cluster labels,Latitude,Longitude
0,Admiralenbuurt,0.0,0,52.372734,4.856363
1,Amsteldorp,0.0,0,52.360540,4.905160
2,Amsterdam Oud-West,0.0,0,52.365390,4.870220
3,Amsterdam Oud-Zuid,0.0,0,52.352350,4.877880
4,Amsterdam Science Park,0.0,0,52.354320,4.958030


In [39]:
amd_merged.sort_values(['Cluster labels'],inplace=True)
amd_merged

,Neighbourhoods,Shopping Mall,Cluster labels,Latitude,Longitude
0,Admiralenbuurt,0.000000,0,52.372734,4.856363
74,Prinses Irenebuurt,0.000000,0,52.341769,4.875224
73,Plantage,0.000000,0,52.366620,4.911890
72,Overtoomse Veld,0.000000,0,52.363540,4.836800
71,Overtoombuurt,0.000000,0,52.359582,4.862307
70,Overhoeks,0.000000,0,52.391680,4.905780
69,Oude Pijp,0.000000,0,52.356250,4.890570
68,Oud-Oost,0.000000,0,52.360130,4.925320
67,Oud Osdorp,0.000000,0,52.356360,4.790600
66,Osdorp,0.000000,0,52.356360,4.790600


<h4>Finally let's visualize the resulting clusters

In [40]:
cluster_map = folium.Map(location=[latitude,longitude],zoom_start=10)

x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for lat,lon,cluster,neigh in zip(amd_merged['Latitude'],amd_merged['Longitude'],amd_merged['Cluster labels'],amd_merged['Neighbourhoods']):
    label = folium.Popup(str(neigh)+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        color= rainbow[cluster-1],
        fill=True,
        popup=label,
        fill_opacity=0.7
    ).add_to(cluster_map)
cluster_map

<h3>8. Examine Clusters

<h5>Cluster 0

In [43]:
amd_merged[amd_merged['Cluster labels']==0]

,Neighbourhoods,Shopping Mall,Cluster labels,Latitude,Longitude
0,Admiralenbuurt,0.00,0,52.372734,4.856363
74,Prinses Irenebuurt,0.00,0,52.341769,4.875224
73,Plantage,0.00,0,52.366620,4.911890
72,Overtoomse Veld,0.00,0,52.363540,4.836800
71,Overtoombuurt,0.00,0,52.359582,4.862307
70,Overhoeks,0.00,0,52.391680,4.905780
69,Oude Pijp,0.00,0,52.356250,4.890570
68,Oud-Oost,0.00,0,52.360130,4.925320
67,Oud Osdorp,0.00,0,52.356360,4.790600
66,Osdorp,0.00,0,52.356360,4.790600


<h5>Cluster 1

In [44]:
amd_merged[amd_merged['Cluster labels']==1]

,Neighbourhoods,Shopping Mall,Cluster labels,Latitude,Longitude
10,Buiksloot,0.111111,1,52.408070,4.918960
50,Molenwijk,0.083333,1,52.421014,4.890155
29,Gaasperdam,0.090909,1,52.296800,4.978120


<h5>Cluster 2

In [45]:
amd_merged[amd_merged['Cluster labels']==2]

,Neighbourhoods,Shopping Mall,Cluster labels,Latitude,Longitude
11,Buikslotermeer,0.029412,2,52.39639,4.93539
12,Buitenveldert,0.032258,2,52.33093,4.87518
46,Landelijk Noord,0.043478,2,52.40072,4.93527
84,Staatsliedenbuurt,0.027778,2,52.38005,4.87149
88,Transvaalbuurt,0.037037,2,52.39308,4.64555
